In [1]:
from facebook_business.api import FacebookAdsApi
from facebook_business.adobjects.adaccount import AdAccount
import pandas as pd
import numpy as np
import re
import gspread
import datetime


from gspread_dataframe import set_with_dataframe

gc = gspread.service_account(filename="./credentials.json")
from sqlalchemy import create_engine
import re

from gspread_dataframe import set_with_dataframe
from pipedrive.client import Client
import environ
env = environ.Env()
database_url = env.str(
    var="DATABASE_URL", default="postgres://postgres:@postgres:5432/postgres"
)
engine = create_engine(database_url, echo=False)
client = Client(domain='https://companydomain.pipedrive.com/')
client.set_api_token('c70f38803bb436089bce792681cd178e8459c86e')

In [2]:
my_app_id = '250119386649304'
my_app_secret = 'bba6aa02d11a3eb62e3b58bad8db7cde'
my_access_token = 'EAADje3UxUtgBAAi6ilZBKGoh3cBqiPj8N0X8XIcsLkCF3WnZABdIaFY8p95l1C6qjRPHL5Aa0gEZCMlcYpCUGujWBgXDnWLJvbvKNVPk8Sclzklsx59eOSjaGUq8UWvegOHq8OZBGyfEUjtcnPBAyYpC4gjYoj9pFZAu6V3LeyeYoFL6sZCDtLVBU9c2dHygMZD'
FacebookAdsApi.init(my_app_id, my_app_secret, my_access_token)

ad_account = AdAccount('act_3386773934749695')

campaigns = ad_account.get_campaigns(fields=['id', 'status', 'lifetime_budget', 'objective'])

params = {'level': 'campaign', 'date_preset': "lifetime"}

fields = ['campaign_name', 'campaign_id', 'reach', 'impressions', 'frequency', 'clicks', 'cpc', 'cpm', 'cpp', 'ctr',
          'spend', 'unique_clicks', 'unique_ctr', 'date_start', 'date_stop']
insights = ad_account.get_insights(params=params, fields=fields)

ins = pd.DataFrame(insights)
cp = pd.DataFrame(campaigns)

cp.rename(columns={"id": "campaign_id"}, inplace=True)
nn = cp.merge(ins, 'inner', on='campaign_id')

nn.head()

,campaign_id,lifetime_budget,objective,status,campaign_name,clicks,cpc,cpm,cpp,ctr,date_start,date_stop,frequency,impressions,reach,spend,unique_clicks,unique_ctr
0,23847019006310634,NaN,VIDEO_VIEWS,ACTIVE,PRO:EMEMO_OBJ:VIDEO_CN:NORM_AW:VIDEO_CONT:FEAT...,196,5.303724,74.358369,76.267792,1.402003,2020-09-22,2021-02-22,1.025679,13980,13630,1039.53,191,1.401321
1,23847018812630634,NaN,CONVERSIONS,PAUSED,PRO:ETAX_OBJ:CONV_CN:NORM_AW:SING_CONT:3BENEFITS,91,5.819341,122.018433,150.700057,2.096774,2020-09-22,2021-02-22,1.23506,4340,3514,529.56,81,2.305065
2,23846978677040634,NaN,LINK_CLICKS,PAUSED,PRO:EMEM_OBJ:TRAF_CN:NORM_AW:SING_CONT:4STEPS,2231,1.04407,42.484132,57.349813,4.069089,2020-09-22,2021-02-22,1.349911,54828,40616,2329.32,2009,4.946327
3,23846948768040634,NaN,LINK_CLICKS,PAUSED,PRO:HR_OBJ:TRAF_CN:RECRUIT_AW:SING_CONT:SENIOR...,272,2.002132,22.870942,28.928552,1.142329,2020-09-22,2021-02-22,1.264861,23811,18825,544.58,264,1.40239
4,23846886524310634,NaN,LINK_CLICKS,PAUSED,PRO:ETAX_OBJ:TRAF_CN:NORM_AW:SING_CONT:BOARD,215,2.04493,30.346494,33.017423,1.483987,2020-09-22,2021-02-22,1.088014,14488,13316,439.66,202,1.516972


In [3]:
def load_facebook_business_zcom(date_start=None, date_stop=None):
    my_app_id = '250119386649304'
    my_app_secret = 'bba6aa02d11a3eb62e3b58bad8db7cde'
    my_access_token = 'EAADje3UxUtgBAAi6ilZBKGoh3cBqiPj8N0X8XIcsLkCF3WnZABdIaFY8p95l1C6qjRPHL5Aa0gEZCMlcYpCUGujWBgXDnWLJvbvKNVPk8Sclzklsx59eOSjaGUq8UWvegOHq8OZBGyfEUjtcnPBAyYpC4gjYoj9pFZAu6V3LeyeYoFL6sZCDtLVBU9c2dHygMZD'
    FacebookAdsApi.init(my_app_id, my_app_secret, my_access_token)

    ad_account = AdAccount("act_3386773934749695")

    campaigns = ad_account.get_campaigns(fields=["id", "status", "objective"])

    if date_start and date_stop:
        p_set = {
            "level": "ad",
            "time_range": {"since": str(date_start), "until": str(date_stop)},
        }
    else:
        p_set = {"level": "ad", "date_preset": "yesterday"}
    f_set = [
        "adset_name",
        "ad_id",
        "ad_name",
        "campaign_id",
        "campaign_name",
        "reach",
        "impressions",
        "frequency",
        "clicks",
        "cpc",
        "cpm",
        "cpp",
        "ctr",
        "spend",
        "unique_clicks",
        "actions",
        "inline_post_engagement",
    ]

    a_set = ad_account.get_insights(params=p_set, fields=f_set)
    ase = pd.DataFrame(a_set)
    ccp = pd.DataFrame(campaigns)

    if ase.empty:
        return "data is empty"

    ccp.rename(
        columns={"id": "campaign_id", "objective": "campaign_objective"}, inplace=True
    )
    nn = ase.merge(ccp, "inner", on="campaign_id")
    nn["type"] = "daily"

    if "actions" in nn.columns:
        ac_df = nn[["actions", "ad_id"]]
        action_type = []
        for idx, row in ac_df.iterrows():
            try:
                if len(row["actions"]) > 0:
                    for dt in row["actions"]:
                        dt["ad_id"] = row["ad_id"]
                        action_type.append(dt)
            except:
                pass

        action_type = pd.DataFrame(action_type)
        action_type = action_type.drop(
            action_type[action_type.action_type != "landing_page_view"].index
        )

        tt = nn.join(action_type.set_index("ad_id"), on="ad_id")
        tt = tt.drop(["actions", "action_type"], axis=1).rename(
            columns={
                "value": "landing_page_view",
                "inline_post_engagement": "post_engagement",
            },
            inplace=False,
        )
    else:
        tt = nn

    cn = []
    d1 = {}
    dict_name = {
        "PRO": "Product",
        "CN": "Campaign name",
        "OBJ": "Objective",
        "AW": "Artwork",
        "CONT": "Content",
    }
    for idx, row in tt.iterrows():
        word = str(row["campaign_name"])
        if word.find("_") == -1:
            continue
        split_word = word.split("_")
        for i in range(4):
            name = split_word[i].split(":")
            col = dict_name.get(name[0])
            d1[col] = name[1]
        d1["ad_id"] = row["ad_id"]
        cn.append(d1)
        d1 = {}

    if cn:
        
        cn = pd.DataFrame(cn)
        tt = tt.join(cn.set_index("ad_id"), on="ad_id")

    tt.to_sql("etl_marketing_facebook_zcom", engine, if_exists="append", index=False)

    return tt

In [10]:
def run_old_daily_facebook_zcom():
    start = datetime.datetime.strptime("1-1-2021", "%d-%m-%Y")
    end = datetime.datetime.today()
    date_generated = [
        start + datetime.timedelta(days=x) for x in range(0, (end - start).days)
    ]

    for date in range(len(date_generated)):
        print(
            date_generated[date].strftime("%Y-%m-%d"),
            " ==> ",
            date_generated[date].strftime("%Y-%m-%d"),
        )
        load_facebook_business_zcom(
            date_generated[date].strftime("%Y-%m-%d"),
            date_generated[date].strftime("%Y-%m-%d"),
        )

In [11]:
run_old_daily_facebook_zcom()

2021-01-01  ==>  2021-01-01
2021-01-02  ==>  2021-01-02
2021-01-03  ==>  2021-01-03
2021-01-04  ==>  2021-01-04
2021-01-05  ==>  2021-01-05
2021-01-06  ==>  2021-01-06
2021-01-07  ==>  2021-01-07
2021-01-08  ==>  2021-01-08
2021-01-09  ==>  2021-01-09
2021-01-10  ==>  2021-01-10
2021-01-11  ==>  2021-01-11
2021-01-12  ==>  2021-01-12
2021-01-13  ==>  2021-01-13
2021-01-14  ==>  2021-01-14
2021-01-15  ==>  2021-01-15
2021-01-16  ==>  2021-01-16
2021-01-17  ==>  2021-01-17
2021-01-18  ==>  2021-01-18
2021-01-19  ==>  2021-01-19
2021-01-20  ==>  2021-01-20
2021-01-21  ==>  2021-01-21
2021-01-22  ==>  2021-01-22
2021-01-23  ==>  2021-01-23
2021-01-24  ==>  2021-01-24
2021-01-25  ==>  2021-01-25
2021-01-26  ==>  2021-01-26
2021-01-27  ==>  2021-01-27
2021-01-28  ==>  2021-01-28
2021-01-29  ==>  2021-01-29
2021-01-30  ==>  2021-01-30
2021-01-31  ==>  2021-01-31
2021-02-01  ==>  2021-02-01
2021-02-02  ==>  2021-02-02
2021-02-03  ==>  2021-02-03
2021-02-04  ==>  2021-02-04
2021-02-05  ==>  202

In [ ]:
load_facebook_business_zcom()